# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Hard: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/859c9ca9ede14fdea841be627c412322)
  - Slide: [link](https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW04/HW04.pdf)
  - Data: [link](https://drive.google.com/file/d/1T0RPnu-Sg5eIPwQPfYysipfcz81MnsYe/view?usp=sharing)
  - Video (Chinese): [link](https://www.youtube.com/watch?v=EPerg2UnGaI)
  - Video (English): [link](https://www.youtube.com/watch?v=Gpz6AUvCak0)
  - Solution for downloading dataset fail.: [link](https://drive.google.com/drive/folders/13T0Pa_WGgQxNkqZk781qhc5T9-zfh19e?usp=sharing)

# Download dataset
- Please follow [here](https://drive.google.com/drive/folders/13T0Pa_WGgQxNkqZk781qhc5T9-zfh19e?usp=sharing) to download data
- Data is [here](https://drive.google.com/file/d/1FCjdQkMk9YAIhRkUjbbbv89YO9HhPD8J/view?usp=sharing)

In [11]:
# !gdown --id '1FCjdQkMk9YAIhRkUjbbbv89YO9HhPD8J' --output Dataset.zip
# !unzip Dataset.zip

# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- We randomly select 600 speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [12]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

 
 
class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len
    
        # Load the mapping from speaker neme to their corresponding id. 
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]
    
        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]
    
        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))
    
        # Segmemt mel-spectrogram into "segment_len" frames.
        if len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[n])
        else:
            mel = torch.FloatTensor(mel)
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker
    
    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [13]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = myDataset(data_dir)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

# class Self_Attentive_Pooling(nn.Module):
#     def __init__(self, batch_rep):
#         super(Self_Attentive_Pooling, self).__init__()
#         self.W = nn.Linear(batch_rep, 1)
    
#     def forward(self, batch_rep):
#         """
#         input:
#             batch_rep : size (N, T, H), N: batch size, T: sequence length, H: Hidden dimension
        
#         attention_weight:
#             att_w : size (N, T, 1)
        
#         return:
#             utter_rep: size (N, H)
#         """
#         softmax = F.softmax
#         att_w = softmax(self.W(batch_rep).squeeze(-1), dim=0).unsqueeze(-1)
#         utter_rep = torch.sum(batch_rep * att_w, dim=1)

#         return utter_rep

class Self_Attention_Pooling(nn.Module):
    def __init__(self, dim):
        super(Self_Attention_Pooling, self).__init__()
        self.sap_linear = nn.Linear(dim, dim)
        self.attention = nn.Parameter(torch.FloatTensor(dim, 1))
    
    def forward(self, x):
        """
        input:
            batch_rep : size (N, T, H), N: batch size, T: sequence length, H: Hidden dimension
        
        attention_weight:
            att_w : size (N, T, 1)
        
        return:
            utter_rep: size (N, H)
        """
        x = x.permute(0, 2, 1)
        h = torch.tanh(self.sap_linear(x))
        w = torch.matmul(h, self.attention).squeeze(dim = 2)
        w = F.softmax(w, dim=1).view(x.size(0), x.size(1), 1)
        x = torch.sum(x*w, dim=1)

        return x

class Classifier(nn.Module):
    def __init__(self, config, n_spks=600, dropout=0.25):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, config['dim'])
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
        # self.encoder_layer = nn.TransformerEncoderLayer(
        #   d_model=d_model, dim_feedforward=256, nhead=2
        # )
        self.encoder_layer = ConformerBlock(**config)
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        self.pooling = Self_Attention_Pooling(config['dim'])
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            # nn.Linear(d_model, d_model),
            # nn.ReLU(),
            # nn.BatchNorm1d(config['dim']),
            nn.Linear(config['dim'], n_spks),
        )


    def forward(self, mels):
        """
        args:
        mels: (batch size, length, 40)
        return:
        out: (batch size, n_spks)
        """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        # out = out.transpose(0, 1)
        # mean pooling
        # stats = out.mean(dim=1)
        out = out.permute(1, 2, 0)  
        stats = self.pooling(out)

        # out: (batch, n_spks)
        out = self.pred_layer(stats)
        return out


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [15]:
import math
import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
    """
    Create a schedule with a learning rate that decreases following the values of the cosine function between the
    initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
    initial lr set in the optimizer.

    Args:
        optimizer (:class:`~torch.optim.Optimizer`):
        The optimizer for which to schedule the learning rate.
        num_warmup_steps (:obj:`int`):
        The number of steps for the warmup phase.
        num_training_steps (:obj:`int`):
        The total number of training steps.
        num_cycles (:obj:`float`, `optional`, defaults to 0.5):
        The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
        following a half-cosine).
        last_epoch (:obj:`int`, `optional`, defaults to -1):
        The index of the last epoch when resuming training.

    Return:
        :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """

    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
        max(1, num_training_steps - num_warmup_steps)
        )
        return max(
        0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)


# Model Function
- Model forward function.

In [16]:
import torch


def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)

    loss = criterion(outs, labels)

    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [17]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
        loss=f"{running_loss / (i+1):.2f}",
        accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)


# Main function

In [18]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "save_path": {
        "config1": "./model1.ckpt",
        "config2": "./model2.ckpt",
        "config3": "./model3.ckpt",
    },
    "batch_size": 32,
    "n_workers": 0,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 200000,
    "model_config":{
        "config1":{
            "dim" : 512,
            "dim_head": 64,
            "heads": 8,
            "ff_mult": 4,
            "conv_expansion_factor": 2,
            "conv_kernel_size": 31,
            "attn_dropout" : 0.1,
            "ff_dropout" : 0.1,
            "conv_dropout" : 0.1,
        },
        "config2":{
            "dim" : 512,
            "dim_head": 32,
            "heads": 1,
            "ff_mult": 4,
            "conv_expansion_factor": 2,
            "conv_kernel_size": 31,
            "attn_dropout" : 0.1,
            "ff_dropout" : 0.1,
            "conv_dropout" : 0.1,
        },
        "config3":{
            "dim" : 512,
            "dim_head": 16,
            "heads": 1,
            "ff_mult": 4,
            "conv_expansion_factor": 2,
            "conv_kernel_size": 31,
            "attn_dropout" : 0.1,
            "ff_dropout" : 0.1,
            "conv_dropout" : 0.1,
        },
    },
  }

  return config

# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def train_main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
    model_config,
):
    """Main function."""
    for i in model_config:
        print(f"[Info]: Use Model-{i} now!")
        same_seeds(0)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"[Info]: Use {device} now!")

        train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
        train_iterator = iter(train_loader)
        print(f"[Info]: Finish loading data!",flush = True)

        model = Classifier(model_config[i], n_spks=speaker_num).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = AdamW(model.parameters(), lr=1e-3)
        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        print(f"[Info]: Finish creating model!",flush = True)

        best_accuracy = -1.0
        best_state_dict = None

        pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        for step in range(total_steps):
            # Get data
            try:
                batch = next(train_iterator)
            except StopIteration:
                train_iterator = iter(train_loader)
                batch = next(train_iterator)

            loss, accuracy = model_fn(batch, model, criterion, device)
            batch_loss = loss.item()
            batch_accuracy = accuracy.item()

            # Updata model
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            
            # Log
            pbar.update()
            pbar.set_postfix(
                loss=f"{batch_loss:.2f}",
                accuracy=f"{batch_accuracy:.2f}",
                step=step + 1,
            )

            # Do validation
            if (step + 1) % valid_steps == 0:
                pbar.close()

                valid_accuracy = valid(valid_loader, model, criterion, device)

                # keep the best model
                if valid_accuracy > best_accuracy:
                    best_accuracy = valid_accuracy
                    best_state_dict = model.state_dict()

                pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

            # Save the best model so far.
            if (step + 1) % save_steps == 0 and best_state_dict is not None:
                # torch.save(best_state_dict, save_path)
                torch.save(best_state_dict, save_path[i])
                pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

        pbar.close()


if __name__ == "__main__":
    train_main(**parse_args())


[Info]: Use Model-config1 now!
[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [00:56<00:00, 35.10 step/s, accuracy=0.00, loss=5.99, step=2000]
Valid: 100% 6336/6344 [00:04<00:00, 1503.30 uttr/s, accuracy=0.04, loss=5.68]
Train: 100% 2000/2000 [00:55<00:00, 35.99 step/s, accuracy=0.09, loss=5.47, step=4000]
Valid: 100% 6336/6344 [00:03<00:00, 1600.24 uttr/s, accuracy=0.09, loss=5.11]
Train: 100% 2000/2000 [00:56<00:00, 35.54 step/s, accuracy=0.22, loss=4.50, step=6000]
Valid: 100% 6336/6344 [00:03<00:00, 1630.22 uttr/s, accuracy=0.12, loss=4.76]
Train: 100% 2000/2000 [00:55<00:00, 36.06 step/s, accuracy=0.12, loss=4.69, step=8000]
Valid: 100% 6336/6344 [00:03<00:00, 1617.35 uttr/s, accuracy=0.16, loss=4.40]
Train: 100% 2000/2000 [00:55<00:00, 35.96 step/s, accuracy=0.06, loss=4.26, step=1e+4]
Valid: 100% 6336/6344 [00:03<00:00, 1617.35 uttr/s, accuracy=0.18, loss=4.26]
Train:   0% 6/2000 [00:00<01:14, 26.76 step/s, accuracy=0.19, loss=3.96, step=1e+4]

Step 10000, best model saved. (accuracy=0.1755)


Train: 100% 2000/2000 [00:55<00:00, 35.95 step/s, accuracy=0.16, loss=4.36, step=12000]
Valid: 100% 6336/6344 [00:03<00:00, 1633.15 uttr/s, accuracy=0.19, loss=4.12]
Train: 100% 2000/2000 [00:55<00:00, 35.87 step/s, accuracy=0.22, loss=3.82, step=14000]
Valid: 100% 6336/6344 [00:03<00:00, 1636.09 uttr/s, accuracy=0.20, loss=4.02]
Train: 100% 2000/2000 [00:56<00:00, 35.09 step/s, accuracy=0.19, loss=4.41, step=16000]
Valid: 100% 6336/6344 [00:03<00:00, 1635.25 uttr/s, accuracy=0.23, loss=3.89]
Train: 100% 2000/2000 [00:55<00:00, 35.88 step/s, accuracy=0.06, loss=4.12, step=18000]
Valid: 100% 6336/6344 [00:03<00:00, 1623.96 uttr/s, accuracy=0.23, loss=3.84]
Train: 100% 2000/2000 [00:55<00:00, 35.96 step/s, accuracy=0.22, loss=3.86, step=2e+4] 
Valid: 100% 6336/6344 [00:03<00:00, 1625.21 uttr/s, accuracy=0.23, loss=3.87]
Train:   0% 7/2000 [00:00<01:06, 29.94 step/s, accuracy=0.44, loss=3.20, step=2e+4]

Step 20000, best model saved. (accuracy=0.2322)


...

Step 180000, best model saved. (accuracy=0.3106)


Train: 100% 2000/2000 [00:56<00:00, 35.71 step/s, accuracy=0.41, loss=2.86, step=182000]
Valid: 100% 6336/6344 [00:03<00:00, 1596.62 uttr/s, accuracy=0.30, loss=3.50]
Train: 100% 2000/2000 [00:54<00:00, 36.55 step/s, accuracy=0.44, loss=2.73, step=184000]
Valid: 100% 6336/6344 [00:03<00:00, 1592.61 uttr/s, accuracy=0.31, loss=3.46]
Train: 100% 2000/2000 [00:54<00:00, 36.47 step/s, accuracy=0.34, loss=3.30, step=186000]
Valid: 100% 6336/6344 [00:03<00:00, 1585.86 uttr/s, accuracy=0.30, loss=3.56]
Train: 100% 2000/2000 [00:54<00:00, 36.48 step/s, accuracy=0.66, loss=1.94, step=188000]
Valid: 100% 6336/6344 [00:03<00:00, 1603.87 uttr/s, accuracy=0.31, loss=3.48]
Train: 100% 2000/2000 [00:54<00:00, 36.48 step/s, accuracy=0.53, loss=2.21, step=190000]
Valid: 100% 6336/6344 [00:03<00:00, 1603.47 uttr/s, accuracy=0.30, loss=3.53]
Train:   0% 7/2000 [00:00<01:09, 28.69 step/s, accuracy=0.53, loss=2.63, step=190007]

Step 190000, best model saved. (accuracy=0.3106)


Train: 100% 2000/2000 [00:54<00:00, 36.48 step/s, accuracy=0.53, loss=2.34, step=192000]
Valid: 100% 6336/6344 [00:03<00:00, 1590.62 uttr/s, accuracy=0.31, loss=3.50]
Train: 100% 2000/2000 [00:54<00:00, 36.46 step/s, accuracy=0.41, loss=2.57, step=194000]
Valid: 100% 6336/6344 [00:03<00:00, 1594.21 uttr/s, accuracy=0.30, loss=3.52]
Train: 100% 2000/2000 [00:54<00:00, 36.49 step/s, accuracy=0.31, loss=3.12, step=196000]
Valid: 100% 6336/6344 [00:03<00:00, 1605.90 uttr/s, accuracy=0.30, loss=3.49]
Train: 100% 2000/2000 [00:54<00:00, 36.49 step/s, accuracy=0.56, loss=2.31, step=198000]
Valid: 100% 6336/6344 [00:03<00:00, 1584.27 uttr/s, accuracy=0.31, loss=3.51]
Train: 100% 2000/2000 [00:54<00:00, 36.42 step/s, accuracy=0.50, loss=2.85, step=2e+5]  
Valid: 100% 6336/6344 [00:03<00:00, 1603.47 uttr/s, accuracy=0.31, loss=3.51]
Train:   0% 0/2000 [00:00<?, ? step/s]

Step 200000, best model saved. (accuracy=0.3106)


# Inference

## Dataset of inference

In [19]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        testdata_path = Path(data_dir) / "testdata.json"
        metadata = json.load(testdata_path.open())
        self.data_dir = data_dir
        self.data = metadata["utterances"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        return feat_path, mel


def inference_collate_batch(batch):
    """Collate a batch of data."""
    feat_paths, mels = zip(*batch)

    return feat_paths, torch.stack(mels)


## Main funcrion of Inference

In [20]:
import json
import csv
from pathlib import Path
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
    """arguments"""
    config = {
        "data_dir": "./Dataset",
        "model_path": {
            "config1":"./model1.ckpt",
            "config2":"./model2.ckpt",
            "config3":"./model3.ckpt",
        },
        "output_path": {
            "config1":"./output1.csv",
            "config2":"./output2.csv",
            "config3":"./output3.csv",
            "config4":"./ensemble_output.csv",
        },
        "model_config":{
            "config1":{
                "dim" : 512,
                "dim_head": 64,
                "heads": 8,
                "ff_mult": 4,
                "conv_expansion_factor": 2,
                "conv_kernel_size": 31,
                "attn_dropout" : 0.1,
                "ff_dropout" : 0.1,
                "conv_dropout" : 0.1,
            },
            "config2":{
                "dim" : 512,
                "dim_head": 32,
                "heads": 1,
                "ff_mult": 4,
                "conv_expansion_factor": 2,
                "conv_kernel_size": 31,
                "attn_dropout" : 0.1,
                "ff_dropout" : 0.1,
                "conv_dropout" : 0.1,
            },
            "config3":{
                "dim" : 512,
                "dim_head": 16,
                "heads": 1,
                "ff_mult": 4,
                "conv_expansion_factor": 2,
                "conv_kernel_size": 31,
                "attn_dropout" : 0.1,
                "ff_dropout" : 0.1,
                "conv_dropout" : 0.1,
            },
        },
    }
    return config


def test_main(
    data_dir,
    model_path,
    output_path,
    model_config,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        num_workers=0,
        collate_fn=inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!",flush = True)

    speaker_num = len(mapping["id2speaker"])
    # model = Classifier(n_spks=speaker_num).to(device)
    # model.load_state_dict(torch.load(model_path))
    model1 = Classifier(model_config["config1"], n_spks=speaker_num).to(device)
    model1.load_state_dict(torch.load(model_path['config1']))
    model1.eval()
    model2 = Classifier(model_config["config2"], n_spks=speaker_num).to(device)
    model2.load_state_dict(torch.load(model_path['config2']))
    model2.eval()
    model3 = Classifier(model_config["config3"], n_spks=speaker_num).to(device)
    model3.load_state_dict(torch.load(model_path['config3']))
    model3.eval()
    print(f"[Info]: Finish creating model!",flush = True)
#----------------------------------------------------------------------
# Model-1
#----------------------------------------------------------------------
    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            outs = model1(mels)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])
    
    with open(output_path["config1"], 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)
#----------------------------------------------------------------------
# Model-2
#----------------------------------------------------------------------
    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            outs = model2(mels)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])
    
    with open(output_path["config2"], 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)
#----------------------------------------------------------------------
# Model-3
#----------------------------------------------------------------------
    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            outs = model3(mels)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])
    
    with open(output_path["config3"], 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)
#----------------------------------------------------------------------
# Ensemble-Model
#----------------------------------------------------------------------
    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            # outs = model(mels)
            outs1 = model1(mels)
            outs2 = model2(mels)
            outs3 = model3(mels)
            outs = (outs1+outs2+outs3) / 3
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])
    
    with open(output_path["config4"], 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


if __name__ == "__main__":
    test_main(**parse_args())


[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


100%|██████████| 6000/6000 [01:04<00:00, 93.64it/s] 
